# 豆瓣电影情感分析

无意中看到了这个数据集，正好练一练NLP。我将使用该数据集建立NLP的模型并进行测试，本实验暂时分为以下几部分：
- NN
- LSTM

由于kaggle线上环境的计算能力有限，若使用数据集的全部数据需要训练非常久的时间，因此线上模型的一些参数设置仅仅是为了快速跑完模型和简单看下效果，不具有实用的意义。我将附上我在线下测试的一些图片和信息。

线下使用了完整的数据集，实验的准确率为90%左右：
![](https://upload-images.jianshu.io/upload_images/15406304-8e87834cdd967e70.png?imageMogr2/auto-orient/strip%7CimageView2/2/w/1240)

线下电脑的配置如下：
![](https://upload-images.jianshu.io/upload_images/15406304-d1031654488a5dc3.png?imageMogr2/auto-orient/strip%7CimageView2/2/w/1240)
![](https://upload-images.jianshu.io/upload_images/15406304-7d4426c0b3558251.png?imageMogr2/auto-orient/strip%7CimageView2/2/w/1240)

## 1、NN

In [1]:
import warnings
warnings.filterwarnings('ignore')
!pip install jieba
!wget -nc "https://codeload.github.com/weiyunchen/nlp/zip/master"
!unzip -o master
import pandas as pd
import os
import time
from tqdm import tqdm_notebook
import re
import time
import copy
import random
import jieba
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import lr_scheduler


--2019-04-20 16:23:40--  https://codeload.github.com/weiyunchen/nlp/zip/master
Resolving codeload.github.com (codeload.github.com)... 192.30.253.120, 192.30.253.121
Connecting to codeload.github.com (codeload.github.com)|192.30.253.120|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘master’

master                  [ <=>                ]   6.27K  --.-KB/s    in 0.003s  

2019-04-20 16:23:41 (1.81 MB/s) - ‘master’ saved [6416]

Archive:  master
862c28a880992c24fa68ff56a0e082d9bddb1af1
   creating: nlp-master/
  inflating: nlp-master/stopwords.txt  


In [2]:
data = pd.read_csv('../input/DMSC.csv', index_col=0)

# 按评分分成两类，1分2分为负面评价,345正面
data['Star']=((data.Star+0.5)/3.5+1).astype(int)

In [3]:
Movie = data['Movie_Name_CN'].value_counts()
Movie

疯狂动物城     137511
大圣归来      133393
后会无期      120200
寻龙诀       113687
你的名字      113260
夏洛特烦恼     109162
釜山行       102876
爱乐之城       96620
西游伏妖篇      91452
小时代1       88903
泰囧         85677
大鱼海棠       83692
长城         83173
西游降魔篇      79962
复仇者联盟      78281
美人鱼        73882
七月与安生      68359
美国队长3      64410
变形金刚4      58746
复仇者联盟2     54153
十二生肖       46233
九层妖塔       44366
小时代3       41152
左耳         39802
湄公河行动      35093
栀子花开       30475
何以笙箫默      26797
钢铁侠1       23739
Name: Movie_Name_CN, dtype: int64

In [4]:
sample_df = data.groupby(['Movie_Name_CN', 'Star']).apply(
    lambda x: x.sample(n=int(2125056/(28*200)), replace=True, random_state=0))

sample_df.shape

(21224, 9)

In [5]:
from sklearn.model_selection import train_test_split


comments = sample_df.values[:, 7]
star = sample_df.values[:, 6]

x_train, x_test, y_train, y_test, = train_test_split(
    comments, star, test_size=0.2, random_state=0)

len(y_train), len(y_test), len(x_train), len(x_test)

(16979, 4245, 16979, 4245)

In [6]:
# 清理非中文字符
def clean_str(line):
    line.strip('\n')
    line = re.sub(r"[^\u4e00-\u9fff]", "", line)
    line = re.sub(
        "[0-9a-zA-Z\-\s+\.\!\/_,$%^*\(\)\+(+\"\')]+|[+——！，。？、~@#￥%……&*（）<>\[\]:：★◆【】《》;；=?？]+", "", line)
    return line.strip()


# 加载停用词
with open('nlp-master/stopwords.txt') as f:
    stopwords = [line.strip('\n') for line in f.readlines()]


def cut(data, labels, stopwords):
    result = []
    new_labels = []
    for index in tqdm_notebook(range(len(data))):
        comment = clean_str(data[index])
        label = labels[index]
        # 分词
        seg_list = jieba.cut(comment, cut_all=False, HMM=True)
        seg_list = [x.strip('\n')
                    for x in seg_list if x not in stopwords and len(x) > 1]
        if len(seg_list) > 1:
            result.append(seg_list)
            new_labels.append(label)
    # 返回分词结果和对应的标签
    return result, new_labels

# 分别对训练数据和测试数据分词
train_cut_result, train_labels = cut(x_train, y_train, stopwords)
test_cut_result, test_labels = cut(x_test, y_test, stopwords)

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.917 seconds.
Prefix dict has been built succesfully.


In [7]:
# TfidfVectorizer 传入原始文本
train_data = [' '.join(x) for x in train_cut_result]
test_data = [' '.join(x) for x in test_cut_result]

n_dim = 20000

# 数据的TF-IDF信息计算
# sublinear_tf=True 时生成一个近似高斯分布的特征，可以提高大概1~2个百分点
vectorizer = TfidfVectorizer(
    max_features=n_dim, smooth_idf=True, sublinear_tf=True)

# 对训练数据训练
train_vec_data = vectorizer.fit_transform(train_data)

# 训练完成之后对测试数据转换
test_vec_data = vectorizer.transform(test_data)

In [8]:
vectorizer.get_feature_names()[:10]

['一丁点', '一万', '一万个', '一万倍', '一万年', '一万次', '一万步', '一万遍', '一下下', '一下子']

In [9]:
# 输出的类别为 2
n_categories = 2
# 学习率过大会导致 loss 震荡
learning_rate = 0.001
# 损失函数
criterion = nn.CrossEntropyLoss()
# 迭代次数
epochs = 3
# 每次迭代同时加载的个数
batch_size = 100

In [10]:
class TxtDataset(Dataset):
    def __init__(self, VectData, labels):
        # 传入初始数据，特征向量和标签
        self.VectData = VectData
        self.labels = labels

    def __getitem__(self, index):
        # DataLoader 会根据 index 获取数据
        # toarray() 是因为 VectData 是一个稀疏矩阵，如果直接使用 VectData.toarray() 占用内存太大，勿尝试
        return self.VectData[index].toarray(), self.labels[index]-1

    def __len__(self):
        return len(self.labels)

# 线下内存足够大可以考虑增大 num_workers，并行读取数据
# 加载训练数据集
train_dataset = TxtDataset(train_vec_data, train_labels)
train_dataloader = DataLoader(train_dataset,
                              batch_size=batch_size,
                              shuffle=True,
                              num_workers=1
                              )
# 加载测试数据集
test_dataset = TxtDataset(test_vec_data, test_labels)
test_dataloader = DataLoader(test_dataset,
                             batch_size=batch_size,
                             shuffle=False,
                             num_workers=1
                             )

In [11]:
class TxtModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(TxtModel, self).__init__()
        self.classifier = nn.Sequential(
            nn.Linear(input_size, 1024),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(1024, 1024),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(1024, 1024),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(1024, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(512, output_size)
        )

    def forward(self, x):
        output = self.classifier(x.double())
        return output.squeeze(1)

In [12]:
# 定义模型和优化器
model = TxtModel(n_dim, 2)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# 每两代衰减学习率
exp_lr_scheduler = lr_scheduler.StepLR(
    optimizer, step_size=int(epochs/2), gamma=0.1)
 
model = model.double()

# 保存准确度最高的模型
best_model = copy.deepcopy(model)
best_accuracy = 0.0

for epoch in range(epochs):
    exp_lr_scheduler.step()
    model.train()
    loss_total = 0
    st = time.time()
    # train_dataloader 加载数据集
    for data, label in tqdm_notebook(train_dataloader):
        output = model(data)
        # 计算损失
        loss = criterion(output, label)
        optimizer.zero_grad()
        # 反向传播
        loss.backward()
        optimizer.step()
        loss_total += loss.item()

    # 输出损失、训练时间等
    print('epoch {}/{}:'.format(epoch, epochs))
    print('training loss: {}, time resumed {}s'.format(
        loss_total/len(train_dataset), time.time()-st))

    model.eval()

    loss_total = 0
    st = time.time()

    correct = 0
    for data, label in test_dataloader:
        output = model(data)
        loss = criterion(output, label)
        loss_total += loss.item()

        _, predicted = torch.max(output.data, 1)
        correct += (predicted == label).sum().item()
    # 如果准确度取得最高，则保存准确度最高的模型
    if correct/len(test_dataset) > best_accuracy:
        best_model = copy.deepcopy(model)

    print('testing loss: {}, time resumed {}s, accuracy: {}'.format(
        loss_total/len(test_dataset), time.time()-st, correct/len(test_dataset)))


epoch 0/3:
training loss: 0.006082983125276355, time resumed 147.52975392341614s
testing loss: 0.0055300475019942116, time resumed 6.537140130996704s, accuracy: 0.7306179775280899



epoch 1/3:
training loss: 0.0036067645759347775, time resumed 146.555602312088s
testing loss: 0.006132205310615474, time resumed 6.523931980133057s, accuracy: 0.7308988764044944



epoch 2/3:
training loss: 0.00321300295739442, time resumed 145.78797507286072s
testing loss: 0.0061692404497828745, time resumed 6.555363178253174s, accuracy: 0.7334269662921349


### 测试模型

In [13]:
import json
import requests
# 26266893 为国产科幻佳作《流浪地球》，在此以《流浪地球》的影评为例
res = requests.get(
    'https://api.douban.com/v2/movie/subject/26266893/comments?apikey=0df993c66c0c636e29ecbb5344252a4a')
comments = json.loads(res.content.decode('utf-8'))['comments']
comments

[{'rating': {'max': 5, 'value': 1.0, 'min': 0},
  'useful_count': 70177,
  'author': {'uid': 'duduxiongzhifu',
   'avatar': 'https://img3.doubanio.com/icon/u2201715-15.jpg',
   'signature': '谁来拧动拧发条鸟的发条',
   'alt': 'https://www.douban.com/people/duduxiongzhifu/',
   'id': '2201715',
   'name': '嘟嘟熊之父 \U0001f9f8'},
  'subject_id': '26266893',
  'content': '还能更土更儿戏一点吗？毫无思考仅靠煽动，毫无敬畏仅余妄想。好的科幻片应该首先承认人类的无知，并跳出人类的视角去看待人与宇宙的关系，而不是一头扎入狭隘的家庭纠纷与大国情怀中自作聪明自我感动。被吹到不行的特效如同导演抡圆了膀子朝观众脸上砸各种金银珠宝，闪到不行但全无美感。有人说作为中国第一部硬科幻电影，不要跟美国比，只想说这宣传攻势票房体量已经超越大多数好莱坞商业大片了，凭啥不能和人家比？所以评价《流浪地球》很简单，你把片中所有角色换成美国人，然后再想想自己愿意打几星。',
  'created_at': '2019-01-28 22:06:27',
  'id': '1646653503'},
 {'rating': {'max': 5, 'value': 4.0, 'min': 0},
  'useful_count': 69017,
  'author': {'uid': 'tjz230',
   'avatar': 'https://img1.doubanio.com/icon/u1005928-127.jpg',
   'signature': '',
   'alt': 'https://www.douban.com/people/tjz230/',
   'id': '1005928',
   'name': '影志'},
  'subject_id': '26266893',
  'content': '电影比预期要更恢弘磅礴，晨昏线过后的永夜、火种计

In [14]:
def predict_comments(comments):
    test_comment = random.choice(comments)
# 选择其中一条分类，并去除非中文字符
    content = clean_str(test_comment['content'])
    rating = test_comment['rating']['value']
# 对评论分词
    seg_list = jieba.cut(content, cut_all=False, HMM=True)
# 去掉停用词和无意义的
    cut_content = ' '.join([x.strip('\n')
                        for x in seg_list if x not in stopwords and len(x) > 1])

# 转化为特征向量
    one_test_data = vectorizer.transform([cut_content])

# 转化为 pytorch 输入的 Tensor 数据，squeeze(0) 增加一个 batch 维度
    one_test_data = torch.from_numpy(one_test_data.toarray()).unsqueeze(0)
# 使用准确度最好的模型预测，softmax 处理输出概率，取得最大概率的下标再加 1 则为预测的标签
    pred = torch.argmax(F.softmax(best_model(one_test_data), dim=1)) + 1
    if rating<3:
        rat='差评1'
    else:
        rat='好评2'
    print('评论内容: ',content)
    print('关键字: ',cut_content)
    print('观众评价: ',rat)
    print('预测评价: ',pred)

In [15]:
for i in range(5):
    print('观后感: ',i)
    print(predict_comments(comments))

观后感:  0
评论内容:  终于轮到我们仰望星空后启示录死亡废墟赛博朋克地下城以及烟波浩渺的末日想象缔造了真正意义上的第一部国产硬科幻拖着地球逃离太阳系的惊艳设定本身便是对家国情怀的宏大投射正应了刘慈欣那句太阳死了人还活着绝不仅仅只是电影工业巨壳下的类型尝试始终荡涤其间的悲壮气息已然具备了史诗级质感这是大刘的脑洞宇宙与电影创作团队精益求精造就的惊喜屈楚萧很带感演活了一个勇敢中二又不失温情的英雄少年期待能成爆款吧这样才有机会等到更多的国产科幻电影或许以后会出现更好的但至今这无疑是最好的
关键字:  终于 仰望 星空 启示录 死亡 废墟 赛博 朋克 地下城 烟波浩渺 末日 想象 缔造 意义 第一部 国产 科幻 地球 逃离 太阳系 惊艳 设定 便是 对家国 情怀 宏大 投射 正应 刘慈欣 那句 太阳 活着 仅仅只是 电影工业 巨壳 类型 尝试 始终 荡涤 其间 悲壮 气息 已然 具备 史诗 质感 脑洞 宇宙 电影 创作 团队 精益求精 造就 惊喜 屈楚 带感 演活 勇敢 中二 不失 温情 英雄 少年 期待 能成 爆款 机会 国产 科幻电影 更好 无疑
观众评价:  好评2
预测评价:  tensor(2)
None
观后感:  1
评论内容:  春节期间最大的一场烟花是看你爸为你和地球点燃的宇宙爆炸太朋克了
关键字:  春节 期间 一场 烟花 地球 点燃 宇宙 爆炸 朋克
观众评价:  好评2
预测评价:  tensor(1)
None
观后感:  2
评论内容:  野心远远大于能力的作品大刘小说打底导演用影视化视觉实现了大部分想象敢想敢做远超预期牛但也仅自于此了故事一塌糊涂世界观做的如此粗糙叙事完全不讲逻辑人物动机行为全都莫名其妙不能理解台词在电影里算的上数一数二的烂最恶心的是国人文化自豪情绪被名目张胆刺裸裸的利用成商业行为明明稍微用点心思就可以更好一点偏不导演编剧仿佛在对你说故事差不多得了太深了他们看不懂就是要俗气直白最要紧的是中国人要拯救地球吴京要操翻世界联合政府中国就爱看这个中国人就只配看这样的科幻故事对啊看看这些评论就只配看这样的科幻故事白瞎了这样的制作白瞎了中国人自己的硬科团队
关键字:  野心 远远 大于 能力 作品 小说 打底 导演 影视 视觉 大部分 想象 远超 预期 自于 故事 一塌糊涂 世界观 粗糙 叙事 逻辑 人物 动机 莫名其妙 理解 台词 电

可以看到，我们用简单的多层神经网络设计的模型有一定的预测能力，由于kaggle线上平台的配置有限，我们只是随机抽取了很少的数据来建模。在使用全部200多万条数据建模时，在我自己的电脑上得到的模型预测效果比线上的效果好非常多，准确率可以达到90%左右。后续我们还会建立复杂的模型。